In [2]:
!pip install pandas numpy sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.7/268.7 kB 4.5 MB/s eta 0:00:00a 0:00:01


In [124]:
# from backend import config
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [125]:
# Load the CSV email dataset
data_frame = pd.read_csv('/kaggle/input/jobemailstatus-dataset/emails_dataset.csv')
data_frame = data_frame.sample(frac=1)
data_frame.head(5)

,subject,message,label
3141,Interview Invitation for Your Application,"Dear Candidate, We are excited to invite you t...",2
1839,Congratulations! You've Been Scheduled for an ...,"Hi, Congratulations! You have progressed to th...",2
2285,Application Received - Thank you for applying!,"Dear Applicant, Thank you for applying to our ...",0
2601,Interview Scheduled - Exciting News!,"Hello, We are pleased to schedule an interview...",2
2997,Your Job Application has been submitted,"Dear Applicant, Thank you for applying to our ...",0


In [126]:
data_frame.isna().sum()

subject    0
message    0
label      0
dtype: int64

In [127]:
# Count of each label
data_frame['message'].groupby(data_frame['label']).count()

label
0    819
1    776
2    734
3    822
Name: message, dtype: int64

In [7]:
!pip install nltk

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [128]:
# Download the essential NLTK files for stopwords, punctuations, and lemmatization
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string

# Download necessary resources
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
print('Successfully Executed!!!')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /usr/share/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Successfully Executed!!!


In [129]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
replace /usr/share/nltk_data/corpora/wordnet/lexnames? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [130]:
# Initialize stop words and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function for removing stopwords, lemmaztize and tokenize the data
def clean_text(text):
    # Tokenize text
    tokens = word_tokenize(text.lower())  # Convert to lowercase
    # Remove stopwords and punctuation, then lemmatize
    cleaned_tokens = [
        lemmatizer.lemmatize(word) for word in tokens
        if word not in stop_words and word not in string.punctuation
    ]
    return ' '.join(cleaned_tokens)

In [131]:
# Creating a copy of the original data frame for a separate tokanized data frame
data_frame_tokanized = data_frame.copy()
data_frame_tokanized.head(3)

,subject,message,label
3141,Interview Invitation for Your Application,"Dear Candidate, We are excited to invite you t...",2
1839,Congratulations! You've Been Scheduled for an ...,"Hi, Congratulations! You have progressed to th...",2
2285,Application Received - Thank you for applying!,"Dear Applicant, Thank you for applying to our ...",0


In [132]:
# Apply the function to the 'subject' and 'messages' columns and store in 'tokenized_subject' and 'tokenized_messages'
data_frame_tokanized['tokenized_subject'] = data_frame_tokanized['subject'].apply(clean_text)
data_frame_tokanized['tokenized_messages'] = data_frame_tokanized['message'].apply(clean_text)
data_frame_tokanized = data_frame_tokanized.drop(['subject', 'message'], axis=1)

data_frame_tokanized.head(5)

,label,tokenized_subject,tokenized_messages
3141,2,interview invitation application,dear candidate excited invite interview please...
1839,2,congratulation 've scheduled interview,hi congratulation progressed interview stage p...
2285,0,application received thank applying,dear applicant thank applying company received...
2601,2,interview scheduled exciting news,hello pleased schedule interview kindly see at...
2997,0,job application submitted,dear applicant thank applying company received...


In [133]:
# Concatenate the tokenized_subject and tokenized_messages
data_frame_tokanized['email_content'] = data_frame_tokanized['tokenized_subject'] + data_frame_tokanized['tokenized_messages']

data_frame_tokanized.head(3)

,label,tokenized_subject,tokenized_messages,email_content
3141,2,interview invitation application,dear candidate excited invite interview please...,interview invitation applicationdear candidate...
1839,2,congratulation 've scheduled interview,hi congratulation progressed interview stage p...,congratulation 've scheduled interviewhi congr...
2285,0,application received thank applying,dear applicant thank applying company received...,application received thank applyingdear applic...


In [134]:
data_frame_preprocessed = data_frame_tokanized.copy()
data_frame_preprocessed = data_frame_preprocessed.drop(['tokenized_subject', 'tokenized_messages'], axis=1)
data_frame_preprocessed = data_frame_preprocessed.sample(frac=1)
# Final Data Frame
data_frame_preprocessed.head(5), data_frame_preprocessed.shape, data_frame_preprocessed.info

(      label                                      email_content
 1317      3  application status decisiondear applicant care...
 104       0  appreciate interest joining brightmindhi drew ...
 385       1  next step ux designer assessmentdear taylor pa...
 2887      0  've received applicationhello received applica...
 1198      1  invitation complete assessmentcongratulation p...,
 (3151, 2),
 <bound method DataFrame.info of       label                                      email_content
 1317      3  application status decisiondear applicant care...
 104       0  appreciate interest joining brightmindhi drew ...
 385       1  next step ux designer assessmentdear taylor pa...
 2887      0  've received applicationhello received applica...
 1198      1  invitation complete assessmentcongratulation p...
 ...     ...                                                ...
 335       1  complete data scientist assessmenthi alex ’ ex...
 1621      0  job application submitteddear applicant thank

In [15]:
!pip install torch transformers

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [135]:
# Import Libraries
import pandas as pd
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from transformers import LongformerTokenizer, LongformerForSequenceClassification
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [138]:
# Prepare Data
# Assuming data_frame_preprocessed is already loaded
data_frame_final = data_frame_preprocessed[['label', 'email_content']]

# Split the data into training and test sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data_frame_final['email_content'].tolist(), 
    data_frame_final['label'].tolist(), 
    test_size=0.2, 
    random_state=42
)

In [ ]:
# Tokenize Data using DistilBert
tokenizer_DB = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize the text
train_encodings = tokenizer_DB(train_texts, truncation=True, padding=True, max_length=1024)
test_encodings = tokenizer_DB(test_texts, truncation=True, padding=True, max_length=1024)

In [96]:
# Tokenize Data using Bert
tokenizer_BERT = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text
train_encodings = tokenizer_BERT(train_texts, truncation=True, padding=True, max_length=2048)
test_encodings = tokenizer_BERT(test_texts, truncation=True, padding=True, max_length=2048)

In [139]:
# Tokenize using Longformer tokenizer
tokenizer_LF = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')

# Tokenize the text with max_length of 2048
train_encodings = tokenizer_LF(train_texts, truncation=True, padding=True, max_length=2048)
test_encodings = tokenizer_LF(test_texts, truncation=True, padding=True, max_length=2048)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [140]:
# Create PyTorch Dataset
class EmailDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = EmailDataset(train_encodings, train_labels)
test_dataset = EmailDataset(test_encodings, test_labels)

In [ ]:
# Initialize DistilBert Model
model_DB = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=4)

In [98]:
# Initialize Bert Model
model_BERT = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [141]:
# Initialize LongFormer Model
model_LF = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096', num_labels=4)

pytorch_model.bin:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
# Define Evaluation Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [100]:
# Training Arguments For Bert
training_args_B = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs'
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [51]:
# Training Arguments For DistilBert
training_args_DB = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs'
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [143]:
# Training Arguments For LongFormer
training_args_LF = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs'
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [101]:
# Train the Bert Model
trainer_B = Trainer(
    model=model_BERT,
    args=training_args_B,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer_B.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.000824,1.000000,1.000000,1.000000,1.000000
2,No log,0.000363,1.000000,1.000000,1.000000,1.000000
3,No log,0.000223,1.000000,1.000000,1.000000,1.000000
4,0.038000,0.000159,1.000000,1.000000,1.000000,1.000000
5,0.038000,0.000124,1.000000,1.000000,1.000000,1.000000
6,0.038000,0.000103,1.000000,1.000000,1.000000,1.000000
7,0.000200,0.000090,1.000000,1.000000,1.000000,1.000000
8,0.000200,0.000082,1.000000,1.000000,1.000000,1.000000
9,0.000200,0.000077,1.000000,1.000000,1.000000,1.000000
10,0.000100,0.000076,1.000000,1.000000,1.000000,1.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1580, training_loss=0.012131533281924792, metrics={'train_runtime': 388.0153, 'train_samples_per_second': 64.946, 'train_steps_per_second': 4.072, 'total_flos': 841764937104000.0, 'train_loss': 0.012131533281924792, 'epoch': 10.0})

In [144]:
# Train the LongFormer Model
trainer_LF = Trainer(
    model=model_LF,
    args=training_args_LF,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer_LF.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
Initializing global attention on CLS token...
Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.000340,1.000000,1.000000,1.000000,1.000000
2,No log,0.000160,1.000000,1.000000,1.000000,1.000000
3,No log,0.000102,1.000000,1.000000,1.000000,1.000000
4,0.033800,0.000074,1.000000,1.000000,1.000000,1.000000
5,0.033800,0.000059,1.000000,1.000000,1.000000,1.000000
6,0.033800,0.000049,1.000000,1.000000,1.000000,1.000000
7,0.000100,0.000043,1.000000,1.000000,1.000000,1.000000
8,0.000100,0.000040,1.000000,1.000000,1.000000,1.000000
9,0.000100,0.000037,1.000000,1.000000,1.000000,1.000000
10,0.000100,0.000037,1.000000,1.000000,1.000000,1.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1580, training_loss=0.010762295968052518, metrics={'train_runtime': 3311.6655, 'train_samples_per_second': 7.609, 'train_steps_per_second': 0.477, 'total_flos': 1083050991792000.0, 'train_loss': 0.010762295968052518, 'epoch': 10.0})

In [23]:
# Train the DistilBert Model
trainer_DB = Trainer(
    model=model_DB,
    args=training_args_DB,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer_DB.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.001547,1.000000,1.000000,1.000000,1.000000
2,No log,0.000489,1.000000,1.000000,1.000000,1.000000
3,No log,0.000247,1.000000,1.000000,1.000000,1.000000
4,0.043700,0.000155,1.000000,1.000000,1.000000,1.000000
5,0.043700,0.000110,1.000000,1.000000,1.000000,1.000000
6,0.043700,0.000085,1.000000,1.000000,1.000000,1.000000
7,0.000200,0.000070,1.000000,1.000000,1.000000,1.000000
8,0.000200,0.000062,1.000000,1.000000,1.000000,1.000000
9,0.000200,0.000057,1.000000,1.000000,1.000000,1.000000
10,0.000100,0.000055,1.000000,1.000000,1.000000,1.000000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=1580, training_loss=0.01392982932002154, metrics={'train_runtime': 275.1346, 'train_samples_per_second': 91.592, 'train_steps_per_second': 5.743, 'total_flos': 423807301008000.0, 'train_loss': 0.01392982932002154, 'epoch': 10.0})

In [103]:
# Evaluate the Bert Model
# Evaluate the model on the test dataset
eval_results = trainer_B.evaluate()
print("Evaluation Results:", eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation Results: {'eval_loss': 7.568518776679412e-05, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 3.0233, 'eval_samples_per_second': 208.711, 'eval_steps_per_second': 13.23, 'epoch': 10.0}


In [24]:
# Evaluate the DistilBert Model
# Evaluate the model on the test dataset
eval_results = trainer_DB.evaluate()
print("Evaluation Results:", eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation Results: {'eval_loss': 5.521651837625541e-05, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 1.6714, 'eval_samples_per_second': 377.52, 'eval_steps_per_second': 23.932, 'epoch': 10.0}


In [145]:
# Evaluate the LongFormer Model
# Evaluate the model on the test dataset
eval_results = trainer_LF.evaluate()
print("Evaluation Results:", eval_results)

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Evaluation Results: {'eval_loss': 3.663305687950924e-05, 'eval_accuracy': 1.0, 'eval_f1': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_runtime': 33.4953, 'eval_samples_per_second': 18.838, 'eval_steps_per_second': 1.194, 'epoch': 10.0}


In [158]:
# from transformers import pipeline

# # Load the summarization pipeline
# summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Test with a sample email content
sample_text = """
Showcase Your Tech Passion with Cisco's Assessment Challenge!
Cisco Jobs
Hi Nilay,

It's an absolute pleasure to meet you! Here at Cisco, we're passionate about discovering exceptional talent in innovative ways.

We invite you to take part in our skill assessment challenge for the Software Engineer I (Full Time) United States role. This is your opportunity to shine and show us what you can do!

Start your challenge here: Assessment Link
Your Candidate ID to access the assessment is: 6404400

The assessment should take around 1 Hour 30 Minutes  to complete. Remember, it's important to work independently, so find a quiet space, minimize distractions, and ensure a smooth experience with Google Chrome or Mozilla Firefox.

A few things to note:

	•	If you've recently completed this challenge for another role, you won't need to                    retake it, all you need to do is click the link for the data to integrate.
	•	Even if you don't finish, your results will still be considered.
	•	While the assessment informs our hiring process, it's not a strict pass/fail test.
	•	For the best results, complete the assessment within the next two weeks.

Should any technical issues arise during the test, SHL Technical Support is available via live chat on their website. If issues persist, or you have further questions, reach out to Cisco Recruiting at assessment_help@cisco.com.

After completing the assessment, you can check your application status on the Cisco Job Portal in a week.

We can't wait to see what you bring to the table!

Cheers,
Cisco Join & Connect
#WeAreCisco

Email ID 32792
This address is unattended and cannot help with questions or requests. To receive these auto-notifications, please add donotreply@cisco.avature.net to your address book.
Follow We Are Cisco Instagram Twitter Facebook Linkedin  YouTube
©2024 Cisco and/or its affiliates	Jobs Help  Cisco.com  Privacy Statement  Trademarks

"""

# # Summarize the text
# summary = summarizer(sample_text, max_length=100, min_length=30, do_sample=False)

# email_summary = summary[0]['summary_text']
# print(email_summary)

In [159]:
# Predict on New Text

# Determine the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the device
model_LF.to(device)

# Update the predict_label function to use the same device
def predict_label(text):
    inputs = tokenizer_LF(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    # Move the inputs to the same device as the model
    inputs = {key: value.to(device) for key, value in inputs.items()}
    outputs = model_LF(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=1).item()
    return predicted_label

print("Predicted Label:", predict_label(sample_text))

Predicted Label: 1


In [161]:
# Saving the Fine Tuned Model
model_LF.save_pretrained("./Mail_LongFormer-Classifier-4096")
# Save the tokenizer
tokenizer_LF.save_pretrained("./Mail_LongFormer-Classifier-4096")

('./Mail_LongFormer-Classifier-4096/tokenizer_config.json',
 './Mail_LongFormer-Classifier-4096/special_tokens_map.json',
 './Mail_LongFormer-Classifier-4096/vocab.json',
 './Mail_LongFormer-Classifier-4096/merges.txt',
 './Mail_LongFormer-Classifier-4096/added_tokens.json')

In [162]:
import shutil

# Zip the model directory
shutil.make_archive('./Mail_LongFormer-Classifier-4096', 'zip', './Mail_LongFormer-Classifier-4096')

'/kaggle/working/Mail_LongFormer-Classifier-4096.zip'

In [ ]:
from transformers import LongformerForSequenceClassification, LongformerTokenizer

# Load model and tokenizer from the local path
model_dir = "/kaggle/working/Mail_LongFormer-Classifier-4096.zip"
model_LF = LongformerForSequenceClassification.from_pretrained(model_dir)
tokenizer_LF = LongformerTokenizer.from_pretrained(model_dir)

In [ ]:
# Loading the Fine Tuned Model
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Load the fine-tuned model
loaded_model = DistilBertForSequenceClassification.from_pretrained("fine_tuned_distilbert")
# Load the tokenizer
loaded_tokenizer = DistilBertTokenizer.from_pretrained("fine_tuned_distilbert")

# Move the loaded model to the appropriate device
loaded_model.to(device)